In [34]:
import os
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import pandas as pd

from astropy.io import fits
from astropy.table import Table
from specutils import Spectrum1D
from specutils.fitting import find_lines_threshold
from scipy.optimize import curve_fit
from astropy.stats import sigma_clip
from glob import glob

# Load in data (downloaded in scratch.ipynb using MAST archive API)

In [2]:
# Load in RU Lupi data
spectrum_files = glob(os.path.join('HST','mastDownload','HST','*','*x1dsum.fits'))
spectrum_files

['HST/mastDownload/HST/lbgj02040/lbgj02040_x1dsum.fits',
 'HST/mastDownload/HST/lbgj02050/lbgj02050_x1dsum.fits',
 'HST/mastDownload/HST/lbgj02060/lbgj02060_x1dsum.fits',
 'HST/mastDownload/HST/lbgj02070/lbgj02070_x1dsum.fits',
 'HST/mastDownload/HST/lbgj02080/lbgj02080_x1dsum.fits',
 'HST/mastDownload/HST/lbgj02090/lbgj02090_x1dsum.fits',
 'HST/mastDownload/HST/lbgj020a0/lbgj020a0_x1dsum.fits',
 'HST/mastDownload/HST/lbgj020b0/lbgj020b0_x1dsum.fits',
 'HST/mastDownload/HST/lbgj53040/lbgj53040_x1dsum.fits',
 'HST/mastDownload/HST/lbgj53050/lbgj53050_x1dsum.fits',
 'HST/mastDownload/HST/lbgj53060/lbgj53060_x1dsum.fits',
 'HST/mastDownload/HST/lbgj53070/lbgj53070_x1dsum.fits',
 'HST/mastDownload/HST/lbgj53080/lbgj53080_x1dsum.fits',
 'HST/mastDownload/HST/lbgj53090/lbgj53090_x1dsum.fits',
 'HST/mastDownload/HST/lbgj530a0/lbgj530a0_x1dsum.fits',
 'HST/mastDownload/HST/lbgj530b0/lbgj530b0_x1dsum.fits',
 'HST/mastDownload/HST/leit1c030/leit1c030_x1dsum.fits',
 'HST/mastDownload/HST/leit1c04

In [ ]:
spectrum_header = [fits.getheader(file, ext=0) for file in spectrum_files]
spectrum_x1d = [Table.read(file, hdu=1) for file in spectrum_files]

In [10]:
date = [header['DATE'] for header in spectrum_header]
observing_run = [header['ROOTNAME'] for header in spectrum_header]
instrument = [header['INSTRUME'] for header in spectrum_header]
instrument_filter = [header['OPT_ELEM'] for header in spectrum_header]

In [ ]:
#from each spectrum, function to get specific lines OR specific transitions from France emission_lines.csv

lines = pd.read_csv('HST/emission_lines.csv', skiprows=2)
lab_emission_lines = lines['lab_lambda']
transition_grouped_lines = lines.groupby("[nu', J']")

In [33]:
spectrum_x1d[0]

SEGMENT,EXPTIME,NELEM,WAVELENGTH,FLUX,ERROR,ERROR_LOWER,VARIANCE_FLAT,VARIANCE_COUNTS,VARIANCE_BKG,GROSS,GCOUNTS,NET,BACKGROUND,DQ,DQ_WGT,BACKGROUND_PER_PIXEL
,s,,Angstrom,erg / (Angstrom s cm2),erg / (Angstrom s cm2),erg / (Angstrom s cm2),,,,ct / s,ct,ct / s,ct / s,,,ct / (pix s)
bytes4,float64,int32,float64[16384],float32[16384],float32[16384],float32[16384],float32[16384],float32[16384],float32[16384],float32[16384],float32[16384],float32[16384],float32[16384],int16[16384],float32[16384],float32[16384]
FUVA,489.152,16384,1277.973751578046 .. 1441.2228355636628,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,128 .. 128,0.0 .. 0.0,0.0 .. 0.0
FUVB,489.152,16384,1124.7185766387568 .. 1287.9209568240676,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,128 .. 128,0.0 .. 0.0,0.0 .. 0.0


In [50]:
flux_units = (u.erg/(u.AA*u.s*(u.cm**-2)))

x1d_wavelength = np.concatenate(spectrum_x1d[0]['WAVELENGTH'])*u.AA
wavesort_idx = np.argsort(x1d_wavelength)
x1d_wavelength = x1d_wavelength[wavesort_idx]

x1d_flux = np.concatenate(spectrum_x1d[0]['FLUX'])*flux_units
x1d_fluxerr = np.concatenate(spectrum_x1d[0]['ERROR'])*flux_units

x1d_flux = x1d_flux[wavesort_idx]
x1d_fluxerr = x1d_fluxerr[wavesort_idx]

spectrum = Spectrum1D(flux = x1d_flux, spectral_axis=x1d_wavelength)

x1d_lines = find_lines_threshold(spectrum, noise_factor=3) #noise_factor is the threshold (x*flux error)

x1d_emissions = x1d_lines[x1d_lines['line_type']=='emission']
x1d_emissions


If you want to suppress this warning either type 'specutils.conf.do_continuum_function_check = False' or see http://docs.astropy.org/en/stable/config/#adding-new-configuration-items for other ways to configure the warning. [specutils.analysis.flux]


line_center,line_type,line_center_index
Angstrom,,
float64,str10,int64
1135.4074708546414,emission,1073
1135.4772026901596,emission,1080
1135.5270111441016,emission,1085
1135.5768195980431,emission,1090
1135.6266280519849,emission,1095
1135.6764365059264,emission,1100
1135.7063215782916,emission,1103
1135.746168341445,emission,1107


In [61]:
#Find the emission lines associated with H2 fluorescence
emission_table_idx = []
for l in lab_emission_lines:
    diff = abs(x1d_emissions['line_center'].value-l)
    if min(diff) < 10:
        emission_table_idx.append(np.argmin(diff))

In [62]:
emission_table_idx

[np.int64(2043),
 np.int64(2050),
 np.int64(2086),
 np.int64(2086),
 np.int64(2028),
 np.int64(2029),
 np.int64(2086),
 np.int64(2027),
 np.int64(2031)]